In [1]:
import numpy as np
import pandas as pd
# import twint
# import time
# from datetime import datetime
# from datetime import timedelta, date
# from collections import Counter
# from ast import literal_eval
import json
import os
from os import listdir
from os.path import isfile, join

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)
pd.options.display.float_format = '{:.2f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# import nest_asyncio
# nest_asyncio.apply()

# Parameters

## PATHS

In [141]:
EXISTING_LOOKUPS_PATHS_ROOT = r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Looking Up All Conspiracy Hashtag User Tweets/All Conspiracy Tweeters' Tweets/Split"    # folder which contains folders which contain CT account tweets
EXISTING_LOOKUPS_PATHS = [ EXISTING_LOOKUPS_PATHS_ROOT + f" {split}/" for split in [5,6,7,8,9] ]      # list of folders which contain CT account tweets

NOTEBOOK_TEMPLATE_PATH = r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Re-Split Lookup/Tweet Lookup TEMPLATE.ipynb"    # path containing .json format of jupyter file template for the lookup

NEW_LOOKUPS_PATH_ROOT = r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Looking Up All Conspiracy Hashtag User Tweets/New Split Tweet Lookups/"    # folder which contains folder for each split

NEW_SPLIT_LIST_SAVE_PATH = r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Getting Conspiracy Hashtag Users/New Splits/"    # folder containing all N splits of the master user list

NEW_NOTEBOOKS_PATH_ROOT = r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Looking Up All Conspiracy Hashtag User Tweets/New Split Scripts/"    # where to save the new .ipynb notebooks

MASTER_USER_LIST_PATH = r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/MASTER LIST - All CT Link and Hashtag Users/All CT Link and Hashtag Users.csv"

## Num Splits

In [3]:
NUM_SPLITS = 20

### Create new folders to put splits in

In [239]:
all_id_split_save_paths = []

for split in range(0, NUM_SPLITS):
    
    newpath = fr'{NEW_LOOKUPS_PATH_ROOT}Split {split}' 
    
    all_id_split_save_paths.append(newpath)
    
    # make a new folder
    if not os.path.exists(newpath):
        os.makedirs(newpath)

# Create Split User ID List

1. Find users who have been looked up already
    * Mine
    * Luke's
2. Bring that information into the master list of users
3. Use ARC to find all users in our master list with GeoCov19 locations
    * Do this so we can search them first
4. Split the users into mine and luke's (only returning the users who haven't been searched up)


## Find all users who have been searched

Luke needs to run the following 5 cells and send me the output (the users he's already searched)

In [5]:
def recover_searched_ids(path):
    
    return [int(f.split('_')[0]) for f in listdir(path) if isfile(join(path, f))]
     

In [158]:
def find_searched_users(split_list):
    '''
    input = list of splits (ints) detailing which split of the dataset we're recovering (e.g. I'm doing [5,6,7,8,9])
    output = np.array of all users (hashtag and link) who have been searched for
    '''   
    
    #create a list of all users we've searched for
    all_searched_users = []
    
    for split in split_list:

        split_n_path = EXISTING_LOOKUPS_PATHS_ROOT + " " + str(split)

        searched_users = recover_searched_ids(split_n_path)
        
        all_searched_users += searched_users
        
    return np.array(all_searched_users)

In [159]:
def recreate_search_updated_dataset(split_list):
    '''
    input = list of splits (ints) detailing which split of the dataset we're recovering (e.g. I'm doing [5,6,7,8,9])
    output = dataframe of all users (hashtag and link) which is updated with searched account['Searched'] == 1
    '''   
    all_searched_users = find_searched_users(split_list)
        
    # bring list of searched users into master list
    all_users = pd.read_csv(MASTER_USER_LIST_PATH).set_index('ID')
    
    if 'Searched' not in all_users.columns:
        all_users['Searched'] = 0
        
    if 'Error' not in all_users.columns:
        all_users['Error'] = 0
    
    all_users['Searched'].loc[all_searched_users] = 1

    
    return all_users

In [ ]:
# MASTER_USER_LIST = recreate_search_updated_dataset([5,6,7,8,9])

# MASTER_USER_LIST[['Num CT Tweets - HT', 'Num CT Tweets - LINK']] = MASTER_USER_LIST[['Num CT Tweets - HT', 'Num CT Tweets - LINK']].fillna(0)

# MASTER_USER_LIST.sort_values(by=['Num CT Tweets - HT', 'Num CT Tweets - LINK'], ascending=False, inplace=True)

# Create Splits

In [230]:
mikhael_searched_users = find_searched_users([5,6,7,8,9])
lukes_searched_users = pd.read_csv(r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Re-Split Lookup/users luke has looked up.csv")

In [231]:
ALL_SEARCHED_USERS = np.concatenate([lukes_searched_users['0'].to_numpy(), mikhael_searched_users])

In [232]:
ALL_CT_USERS = pd.read_csv(MASTER_USER_LIST_PATH)

ALL_CT_USERS['Searched'] = 0

ALL_CT_USERS['Error'] = 0

ALL_CT_USERS.set_index('ID', inplace=True)

ALL_CT_USERS.rename(columns={'Unnamed: 0': 'Original Order'})

ALL_CT_USERS['Searched'].loc[ALL_SEARCHED_USERS] = 1

ALL_CT_USERS[['Num CT Tweets - HT', 'Num CT Tweets - LINK']] = ALL_CT_USERS[['Num CT Tweets - HT', 'Num CT Tweets - LINK']].fillna(0)

# ALL_CT_USERS.sort_values(by=['Num CT Tweets - HT', 'Num CT Tweets - LINK'], ascending=False, inplace=True)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [233]:
# create a weighted score to sort CT users by
ALL_CT_USERS['Weighted Num Tweets'] = ALL_CT_USERS['Num CT Tweets - HT']/ALL_CT_USERS['Num CT Tweets - HT'].sum()
ALL_CT_USERS['Weighted Num Links'] = ALL_CT_USERS['Num CT Tweets - LINK']/ALL_CT_USERS['Num CT Tweets - LINK'].sum()
ALL_CT_USERS['Weighted CT Score'] = ALL_CT_USERS['Weighted Num Links'] + ALL_CT_USERS['Weighted Num Tweets']

ALL_CT_USERS.sort_values(['Weighted CT Score'], ascending=False, inplace=True)

# ALL_CT_USERS.drop(['Weighted CT Score', 'Weighted Num Tweets', 'Weighted Num Links'], axis=1, inplace=True)

## Create New Splits

In [234]:
users_to_search = ALL_CT_USERS[ ALL_CT_USERS['Searched']==0 ]

In [235]:
luke_users_to_search = users_to_search[0::2]
mikhael_users_to_search = users_to_search[1::2]

In [236]:
lukes_splits = {split:luke_users_to_search[split::NUM_SPLITS] for split in range(0, NUM_SPLITS)}

In [237]:
mikhaels_splits = {split:mikhael_users_to_search[split::NUM_SPLITS] for split in range(0, NUM_SPLITS)}

In [240]:
for split in range(0, NUM_SPLITS):
    mikhaels_splits[split].to_csv(NEW_SPLIT_LIST_SAVE_PATH + f"Split {split}.csv")

LUKES_SPLIT_PATH = r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Getting Conspiracy Hashtag Users/New Splits for Luke/"
for split in range(0, NUM_SPLITS):
    lukes_splits[split].to_csv(LUKES_SPLIT_PATH + f"Split {split}.csv")


### Figure out new split of users (links)

In [87]:
link_user_df = pd.read_csv(r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Getting Conspiracy Link Users/All Link Users.csv")

In [88]:
link_user_ids = link_user_df['ID']

In [89]:
link_user_ids = np.array_split(link_user_ids, 2)

lukes_link_users = link_user_ids[0]
mikhaels_link_users = link_user_ids[1]

## Putting old and new splits together

In [90]:
luke_final_users = list(lukes_link_users) + list(lukes_og_users)

In [91]:
mikhaels_final_users = list(mikhaels_link_users) + list(mikhaels_og_users)

# Create new Jupyter Notebooks

In [243]:
def gen_split_n_notebook(split):
    
    save_ipynb_path = NEW_NOTEBOOKS_PATH_ROOT + f'Split {split}.ipynb'
    
    save_path_for_this_splits_users = NEW_LOOKUPS_PATH_ROOT + f'Split {split}'
    
    save_path_to_update_searched_users = NEW_SPLIT_LIST_SAVE_PATH + f'Split {split}.csv'
    
    with open(NOTEBOOK_TEMPLATE_PATH, 'r') as fp:
        notebook = json.load(fp)
    
    for cell in notebook['cells']:
        if cell['cell_type']=='code':
            cell['source'] = [x.replace('REPLACE_SPLIT_USER_IDS_PATH', "'" + save_path_to_update_searched_users + "'").replace('REPLACE_PATH_ROOT', "'" + save_path_for_this_splits_users + "/'").replace('REPLACE_WITH_SPLIT_NUMBER', str(split)) for x in cell['source']]
            
    with open(save_ipynb_path, 'w') as fp:
        json.dump(notebook, fp)

In [244]:
for split in range(0, NUM_SPLITS):
    gen_split_n_notebook(split)

# Run New Notebooks

In [229]:
%run r"C:\Users\crackcocaine69xxx\Python Stuff\594\Twint\Looking Up All Conspiracy Hashtag User Tweets\New Split Scripts\Split 0.ipynb"

Exception: File `'r"C:\\Users\\crackcocaine69xxx\\Python.py'` not found.